In [1]:
import requests
from bs4 import BeautifulSoup
import js2xml
import pandas as pd

In [2]:
import pyecharts.options as opts
from pyecharts.charts import Line, Bar, Pie
from pyecharts.commons.utils import JsCode

In [3]:
df = pd.read_csv('pl_data.csv')

In [100]:
df

,name,value,date
0,java,26.492,2001-6-30
1,java,25.028,2001-7-30
2,java,24.660,2001-8-30
3,java,24.817,2001-9-28
4,java,25.676,2001-10-26
...,...,...,...
4531,labview,0.517,2017-10-5
4532,labview,0.464,2017-11-12
4533,labview,0.838,2017-12-9
4534,labview,0.416,2018-1-3


In [ ]:
# 语言总体排名

In [4]:
total_rate = df[df['date'] == '2019-12-6']

In [5]:
def bar_total() -> Bar:
    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )
    c = (
        Bar(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add_xaxis(total_rate['name'].values.tolist())
        .add_yaxis("编程语言排名", total_rate['value'].values.tolist(), 
                   label_opts=opts.LabelOpts(is_show=False),
                  color='plum')
        .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30),),
                        yaxis_opts=opts.AxisOpts(
                            axislabel_opts=opts.LabelOpts(formatter="{value} %"),)
                        )
    )
    return c

In [164]:
bar_total().render_notebook()

In [ ]:
# 图表函数

In [6]:
def line_color_with_js_func(x_data, y_data, bar_y_data, name) -> Line:
    x_data = x_data
    y_data = y_data

    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )
    area_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
    )

    line = (
        Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name="排名指数",
            y_axis=y_data,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="#fff"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="white"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="red", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .extend_axis(
            yaxis=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(formatter="{value}"), interval=5
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="%s 2019" % name,
                pos_bottom="6%",
                pos_left="center",
                title_textstyle_opts=opts.TextStyleOpts(color="#fff", font_size=16),
            ),
            xaxis_opts=opts.AxisOpts(
                type_="category",
                boundary_gap=False,
                axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63", rotate=-15),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=25,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                position="right",
                axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63", formatter="{value} %"),
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
                ),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=15,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            legend_opts=opts.LegendOpts(is_show=True),
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                ]
            )
        )
    )
    
    color_function = {
            "normal": {
                "color": JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                    offset: 0,
                    color: 'rgba(255,100,97,.5)'
                }, {
                    offset: 1,
                    color: 'rgba(0, 77, 167, 1)'
                }], false)"""),
                "barBorderRadius": [30, 30, 30, 30],
                "shadowColor": 'rgb(255,100,97)',
            }}
           
    bar = (
        Bar()
        .add_xaxis(x_data)
        .add_yaxis("增长率", bar_y_data, yaxis_index=1, category_gap="60%")
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=color_function
        )
    )
    line.overlap(bar)
    return line

In [245]:
# Java 2019 走势
java_2019 = df[df['name'] == 'java']

In [243]:
java_x_data = java_2019.iloc[-12:]['date'].values.tolist()
java_y_data = java_2019.iloc[-12:]['value'].values.tolist()
java_2019_rate = list(map(lambda x, y: (x - y), java_2019.iloc[-12:]['value'].values.tolist(), java_2019.iloc[-13:-1]['value'].values.tolist()))

In [251]:
line_color_with_js_func(java_x_data, java_y_data, java_2019_rate, 'Java').render_notebook()

In [246]:
# C 2019 走势
c_2019 = df[df['name'] == 'c']

In [273]:
c_x_data = c_2019.iloc[-12:]['date'].values.tolist()
c_y_data = c_2019.iloc[-12:]['value'].values.tolist()
c_y_data = list(map(lambda x: float('%.3f' % x), c_y_data))
c_2019_rate = list(map(lambda x, y: (x - y), c_2019.iloc[-12:]['value'].values.tolist(), c_2019.iloc[-13:-1]['value'].values.tolist()))

In [274]:
line_color_with_js_func(c_x_data, c_y_data, c_2019_rate, 'C').render_notebook()

In [7]:
# Python 2019 走势
python_2019 = df[df['name'] == 'python']

In [8]:
python_x_data = python_2019.iloc[-12:]['date'].values.tolist()
python_y_data = python_2019.iloc[-12:]['value'].values.tolist()
python_y_data = list(map(lambda x: float('%.3f' % x), python_y_data))
python_2019_rate = list(map(lambda x, y: (x - y), python_2019.iloc[-12:]['value'].values.tolist(), python_2019.iloc[-13:-1]['value'].values.tolist()))

In [9]:
line_color_with_js_func(python_x_data, python_y_data, python_2019_rate, 'Python').render_notebook()

In [291]:
# C++ 2019 走势
cplusplus_2019 = df[df['name'] == 'c++']

In [293]:
cplusplus_x_data = cplusplus_2019.iloc[-12:]['date'].values.tolist()
cplusplus_y_data = cplusplus_2019.iloc[-12:]['value'].values.tolist()
cplusplus_y_data = list(map(lambda x: float('%.3f' % x), cplusplus_y_data))
cplusplus_2019_rate = list(map(lambda x, y: (x - y), cplusplus_2019.iloc[-12:]['value'].values.tolist(), cplusplus_2019.iloc[-13:-1]['value'].values.tolist()))

In [294]:
line_color_with_js_func(cplusplus_x_data, cplusplus_y_data, cplusplus_2019_rate, 'C++').render_notebook()

In [284]:
# C# 2019 走势
csharp_2019 = df[df['name'] == 'c#']

In [287]:
csharp_x_data = csharp_2019.iloc[-12:]['date'].values.tolist()
csharp_y_data = csharp_2019.iloc[-12:]['value'].values.tolist()
csharp_y_data = list(map(lambda x: float('%.3f' % x), csharp_y_data))
csharp_2019_rate = list(map(lambda x, y: (x - y), csharp_2019.iloc[-12:]['value'].values.tolist(), csharp_2019.iloc[-13:-1]['value'].values.tolist()))

In [288]:
line_color_with_js_func(csharp_x_data, csharp_y_data, csharp_2019_rate, 'C#').render_notebook()

In [ ]:
# Visual Basic .Net,JavaScript, PHP, SQL, Swift

In [19]:
# 6-10 名曲线
def line_color_6_10(x_data, y1, y2, y3, y4, y5) -> Line:
    x_data = x_data

    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )
    area_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
    )

    line = (
        Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name=".Net 排名指数",
            y_axis=y1,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="green"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="green"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="green", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="JvaScript 排名指数",
            y_axis=y2,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="blue"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="blue"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="blue", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="PHP 排名指数",
            y_axis=y3,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="tomato"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="tomato"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="tomato", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="SQL 排名指数",
            y_axis=y4,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="violetred"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="violetred"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="violetred", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="Swift 排名指数",
            y_axis=y5,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="#fff"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="white"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="red", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )

        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="",
                pos_bottom="6%",
                pos_left="center",
                title_textstyle_opts=opts.TextStyleOpts(color="#fff", font_size=16),
            ),
            xaxis_opts=opts.AxisOpts(
                type_="category",
                boundary_gap=False,
                axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63", rotate=-15),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=25,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                position="right",
                axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63", formatter="{value} %"),
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
                ),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=15,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            legend_opts=opts.LegendOpts(is_show=True),
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                ]
            )
        )
    )

    return line

In [8]:
vbn_2019 = df[df['name'] == 'visual-basic-dotnet']
javascript_2019 = df[df['name'] == 'javascript']
php_2019 = df[df['name'] == 'php']
sql_2019 = df[df['name'] == 'sql']
swift_2019 = df[df['name'] == 'swift']

In [9]:
vbn_x_data = vbn_2019.iloc[-12:]['date'].values.tolist()
vbn_y_data = vbn_2019.iloc[-12:]['value'].values.tolist()
vbn_y_data = list(map(lambda x: float('%.3f' % x), vbn_y_data))

javascript_y_data = javascript_2019.iloc[-12:]['value'].values.tolist()
javascript_y_data = list(map(lambda x: float('%.3f' % x), javascript_y_data))

php_y_data = php_2019.iloc[-12:]['value'].values.tolist()
php_y_data = list(map(lambda x: float('%.3f' % x), php_y_data))

sql_y_data = sql_2019.iloc[-12:]['value'].values.tolist()
sql_y_data = list(map(lambda x: float('%.3f' % x), sql_y_data))

swift_y_data = swift_2019.iloc[-12:]['value'].values.tolist()
swift_y_data = list(map(lambda x: float('%.3f' % x), swift_y_data))

In [34]:
line_color_6_10(vbn_x_data, vbn_y_data, javascript_y_data, php_y_data, sql_y_data, swift_y_data).render_notebook()

In [ ]:
# 2019 年 11-15 名
# Ruby, delphi-object-pascal, Objective-C, Assembly-Ianguage, Go, 

In [29]:
ruby_2019 = df[df['name'] == 'ruby']
delphi_pascal_2019 = df[df['name'] == 'delphi-object-pascal']
objective_c_2019 = df[df['name'] == 'objective-c']
assembly_language_2019 = df[df['name'] == 'assembly-language']
go_2019 = df[df['name'] == 'go']

In [30]:
ruby_x_data = ruby_2019.iloc[-12:]['date'].values.tolist()
ruby_y_data = ruby_2019.iloc[-12:]['value'].values.tolist()
ruby_y_data = list(map(lambda x: float('%.3f' % x), ruby_y_data))

delphi_pascal_y_data = delphi_pascal_2019.iloc[-12:]['value'].values.tolist()
delphi_pascal_y_data = list(map(lambda x: float('%.3f' % x), delphi_pascal_y_data))

objective_c_y_data = objective_c_2019.iloc[-12:]['value'].values.tolist()
objective_c_y_data = list(map(lambda x: float('%.3f' % x), objective_c_y_data))

assembly_language_y_data = assembly_language_2019.iloc[-12:]['value'].values.tolist()
assembly_language_y_data = list(map(lambda x: float('%.3f' % x), assembly_language_y_data))

go_y_data = go_2019.iloc[-12:]['value'].values.tolist()
go_y_data = list(map(lambda x: float('%.3f' % x), go_y_data))

In [37]:
# 11-15 名曲线
def line_color_11_15(x_data, y1, y2, y3, y4, y5) -> Line:
    x_data = x_data

    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )
    area_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
    )

    line = (
        Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name="Ruby 排名指数",
            y_axis=y1,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="green"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="green"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="green", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="Delphi 排名指数",
            y_axis=y2,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="blue"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="blue"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="blue", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="Objective-C 排名指数",
            y_axis=y3,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="tomato"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="tomato"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="tomato", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="Assembly 排名指数",
            y_axis=y4,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="violetred"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="violetred"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="violetred", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="Go 排名指数",
            y_axis=y5,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="#fff"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="white"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="red", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )

        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="",
                pos_bottom="6%",
                pos_left="center",
                title_textstyle_opts=opts.TextStyleOpts(color="#fff", font_size=16),
            ),
            xaxis_opts=opts.AxisOpts(
                type_="category",
                boundary_gap=False,
                axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63", rotate=-15),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=25,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                position="right",
                axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63", formatter="{value} %"),
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
                ),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=15,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            legend_opts=opts.LegendOpts(is_show=True),
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                ]
            )
        )
    )

    return line

In [38]:
line_color_11_15(ruby_x_data, ruby_y_data, delphi_pascal_y_data, objective_c_y_data, assembly_language_y_data, go_y_data).render_notebook()

In [39]:
# 2019 年 16-20 名
# R, MATLAB, D, Visual-Basic, Perl

In [42]:
r_2019 = df[df['name'] == 'r']
matlab_2019 = df[df['name'] == 'matlab']
d_2019 = df[df['name'] == 'd']
vislual_basic_2019 = df[df['name'] == 'visual-basic']
perl_2019 = df[df['name'] == 'perl']

In [43]:
r_x_data = r_2019.iloc[-12:]['date'].values.tolist()
r_y_data = r_2019.iloc[-12:]['value'].values.tolist()
r_y_data = list(map(lambda x: float('%.3f' % x), r_y_data))

matlab_y_data = matlab_2019.iloc[-12:]['value'].values.tolist()
matlab_y_data = list(map(lambda x: float('%.3f' % x), matlab_y_data))

d_y_data = d_2019.iloc[-12:]['value'].values.tolist()
d_y_data = list(map(lambda x: float('%.3f' % x), d_y_data))

vislual_basic_y_data = vislual_basic_2019.iloc[-12:]['value'].values.tolist()
vislual_basic_y_data = list(map(lambda x: float('%.3f' % x), vislual_basic_y_data))

perl_y_data = perl_2019.iloc[-12:]['value'].values.tolist()
perl_y_data = list(map(lambda x: float('%.3f' % x), perl_y_data))

In [44]:
# 16-20 名曲线
def line_color_16_20(x_data, y1, y2, y3, y4, y5) -> Line:
    x_data = x_data

    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )
    area_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
    )

    line = (
        Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name="R 排名指数",
            y_axis=y1,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="green"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="green"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="green", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="MATLAB 排名指数",
            y_axis=y2,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="blue"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="blue"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="blue", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="D 排名指数",
            y_axis=y3,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="tomato"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="tomato"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="tomato", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="Visual Basic 排名指数",
            y_axis=y4,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="violetred"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="violetred"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="violetred", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .add_yaxis(
            series_name="Perl 排名指数",
            y_axis=y5,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="#fff"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="white"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="red", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )

        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="",
                pos_bottom="6%",
                pos_left="center",
                title_textstyle_opts=opts.TextStyleOpts(color="#fff", font_size=16),
            ),
            xaxis_opts=opts.AxisOpts(
                type_="category",
                boundary_gap=False,
                axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63", rotate=-15),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=25,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                position="right",
                axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63", formatter="{value} %"),
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
                ),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=15,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            legend_opts=opts.LegendOpts(is_show=True),
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                ]
            )
        )
    )

    return line

In [46]:
line_color_16_20(r_x_data, r_y_data, matlab_y_data, d_y_data, vislual_basic_y_data, perl_y_data).render_notebook()

In [ ]:
# 前五语言占比

In [124]:
def pie_rosetype(name, value) -> Pie:
    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )

    c = (
        Pie(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add(
            "",
            [list(z) for z in zip(name, value)],
            radius=["30%", "75%"],
            center=["35%", "50%"],
            rosetype="area",
            label_opts=opts.LabelOpts(is_show=True, formatter="{b}: {c}", color="violetred",),
        )
        .add(
            "",
            [list(z) for z in zip(['五大语言', '其他语言'], [top_5_total, 100-top_5_total])],
            radius=[60, 80],
            center=["75%", "50%"],
            rosetype="radius",
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="五大语言占比"))

    )
    return c

In [49]:
sum_value = df[df['date'] == '2019-12-6']['value'].sum()

In [97]:
top_5_name = ['Java', 'C', 'Python', 'C++', 'C#']
top_java_value = float(df[df['date'] == '2019-12-6'].iloc[0]['value'])
top_c_value = float(df[df['date'] == '2019-12-6'].iloc[1]['value'])
top_python_value = float(df[df['date'] == '2019-12-6'].iloc[2]['value'])
top_cplus_value = float("%.3f" % df[df['date'] == '2019-12-6'].iloc[3]['value'])
top_csharp_value = float(df[df['date'] == '2019-12-6'].iloc[4]['value'])
top_5_value = [top_java_value, top_c_value, top_python_value, top_cplus_value, top_csharp_value]
top_5_total = top_java_value + top_c_value + top_python_value + top_cplus_value + top_csharp_value

In [125]:
pie_rosetype(top_5_name, top_5_value).render_notebook()

In [177]:
top_dotnet_value = float(df[df['date'] == '2019-12-6'].iloc[5]['value'])
top_js_value = float(df[df['date'] == '2019-12-6'].iloc[6]['value'])
top_7_name = ['Java', 'C', 'Python', 'C++', 'C#', 'dotnet', 'JS']
top_7_value = [top_java_value, top_c_value, top_python_value, top_cplus_value, top_csharp_value, top_dotnet_value, top_js_value]
top_7_total = top_java_value + top_c_value + top_python_value + top_cplus_value + top_csharp_value + top_dotnet_value + top_js_value

In [173]:
def pie_rosetype7(name, value) -> Pie:
    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )

    c = (
        Pie(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add(
            "",
            [list(z) for z in zip(name, value)],
            radius=["30%", "75%"],
            center=["35%", "50%"],
            rosetype="area",
            label_opts=opts.LabelOpts(is_show=True, formatter="{b}: {c}", color="violetred",),
        )
        .add(
            "",
            [list(z) for z in zip(['七大语言', '其他语言'], [top_7_total, 100-top_7_total])],
            radius=[60, 80],
            center=["75%", "50%"],
            rosetype="radius",
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="七大语言占比"))

    )
    return c

In [178]:
pie_rosetype7(top_7_name, top_7_value).render_notebook()

In [130]:
def python_own_charts() -> Line:
    x_data = python_x_data
    y_data = python_y_data

    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )
    area_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
    )

    line = (
        Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(
            series_name="排名指数",
            y_axis=y_data,
            is_smooth=True,
            is_symbol_show=True,
            symbol="circle",
            symbol_size=6,
            linestyle_opts=opts.LineStyleOpts(color="#fff"),
            label_opts=opts.LabelOpts(is_show=True, position="top", color="white"),
            itemstyle_opts=opts.ItemStyleOpts(
                color="red", border_color="#fff", border_width=3
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
        )
        .extend_axis(
            yaxis=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(formatter="{value}"), interval=5
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="%s 2019" % 'Python',
                pos_bottom="6%",
                pos_left="center",
                title_textstyle_opts=opts.TextStyleOpts(color="#fff", font_size=16),
            ),
            xaxis_opts=opts.AxisOpts(
                type_="category",
                boundary_gap=False,
                axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63", rotate=-15),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=25,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                position="right",
                axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63", formatter="{value} %"),
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
                ),
                axistick_opts=opts.AxisTickOpts(
                    is_show=True,
                    length=15,
                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                ),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                ),
            ),
            legend_opts=opts.LegendOpts(is_show=True),

            # Graphic
            graphic_opts=[
                opts.GraphicGroup(
                    graphic_item=opts.GraphicItem(
                        left="40%",
                        top="45%",
                        # rotation=JsCode("Math.PI / 4"),
                        bounding="all",
                        right=110,
                        bottom=110,
                        z=100,
                        is_draggable=True,
                    ),
                    children=[
                        opts.GraphicRect(
                            graphic_item=opts.GraphicItem(
                                left="center", top="middle", z=100
                            ),
                            graphic_shape_opts=opts.GraphicShapeOpts(
                                width=190, height=70,
                            ),
                            graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                fill="rgba(0,0,0,0.3)"
                            ),
                        ),
                        opts.GraphicText(
                            graphic_item=opts.GraphicItem(
                                left="center", top="middle", z=100
                            ),
                            graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                                text=JsCode(
                                    "['Python 语言2019年',"
                                    "'全年增长超过 2%',"
                                    "'与前两名的差距',"
                                    "'正在逐步迫近'].join('\\n')"
                                ),
                                font="14px Microsoft YaHei",
                                graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                    fill="#fff"
                                ),
                            ),
                        ),
                        opts.GraphicImage(
                            graphic_item=opts.GraphicItem(
                                id_="logo",
                                right=20,
                                top=20,
                                z=-10,
                                bounding="raw",
                                origin=[75, 75],
                            ),
                            graphic_imagestyle_opts=opts.GraphicImageStyleOpts(
                                image="file:///C:/Work/sanjutou.png",
                                width=250,
                                height=150,
                                opacity=1,
                            ),
                )
                    ],
                )
            ],
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                ]
            )
        )
    )
    
    color_function = {
            "normal": {
                "color": JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                    offset: 0,
                    color: 'rgba(255,100,97,.5)'
                }, {
                    offset: 1,
                    color: 'rgba(0, 77, 167, 1)'
                }], false)"""),
                "barBorderRadius": [30, 30, 30, 30],
                "shadowColor": 'rgb(255,100,97)',
            }}
           
    bar = (
        Bar()
        .add_xaxis(x_data)
        .add_yaxis("增长率", python_2019_rate, yaxis_index=1, category_gap="60%")
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=color_function
        )
    )
    line.overlap(bar)
    return line

In [131]:
python_own_charts().render()

'C:\\Work\\code\\Python\\data_combat\\Annual_Ceremony\\Programming_language\\render.html'

In [115]:
df[df['name'] == 'java']

,name,value,date
0,java,26.492,2001-6-30
1,java,25.028,2001-7-30
2,java,24.660,2001-8-30
3,java,24.817,2001-9-28
4,java,25.676,2001-10-26
...,...,...,...
214,java,16.028,2019-8-5
215,java,16.661,2019-9-9
216,java,16.884,2019-10-5
217,java,16.246,2019-11-3


In [124]:
def line_total() -> Line:
    background_color_js = (
        "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
        "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
    )
    c = (
        Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
        .add_xaxis(df[df['name'] == 'java']['date'].values.tolist())
        .add_yaxis("Java", df[df['name'] == 'java']['value'].values.tolist(), 
                   label_opts=opts.LabelOpts(is_show=False),
                  color='white')
        .add_yaxis("C", df[df['name'] == 'c']['value'].values.tolist(), 
                   label_opts=opts.LabelOpts(is_show=False),
                  color='plum')
        .add_yaxis("Python", df[df['name'] == 'python']['value'].values.tolist(), 
                   label_opts=opts.LabelOpts(is_show=False),
                  color='black')
        .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30),),
                        yaxis_opts=opts.AxisOpts(
                            axislabel_opts=opts.LabelOpts(formatter="{value} %"),)
                        )
    )
    return c

In [125]:
line_total().render_notebook()